In [10]:
# Imports
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [11]:
# Import the Csv's
payments_df = pd.read_csv('/mnt/c/Users/kpali/Downloads/Payments.csv')
clients_df = pd.read_csv('/mnt/c/Users/kpali/Downloads/Clients.csv')

In [12]:
# Shape of each dataset
print(payments_df.shape)
print(clients_df.shape)

(25559, 6)
(1287, 3)


In [13]:
# Data types of each column
print(payments_df.dtypes)
print(clients_df.dtypes)

transaction_id        int64
contract_id           int64
client_id             int64
transaction_date      int64
payment_amt         float64
payment_code         object
dtype: object
client_id                   int64
entity_type                object
entity_year_established     int64
dtype: object


In [14]:
# Number of nulls in columns for each dataframe
payments_df_null_counts = payments_df.isnull().sum()
clients_df_null_counts = clients_df.isnull().sum()
print(payments_df_null_counts)
print(clients_df_null_counts)

transaction_id      0
contract_id         0
client_id           0
transaction_date    0
payment_amt         0
payment_code        0
dtype: int64
client_id                  0
entity_type                0
entity_year_established    0
dtype: int64


In [15]:
# Convert epoch timestamp for transaction_date to a month-year timestamp
payments_df['transaction_date'] = pd.to_datetime(payments_df['transaction_date'], unit = 's')
payments_df['transaction_date'] = payments_df['transaction_date'].dt.strftime('%m-%Y')

In [17]:
# Explore contents of columns + records in each column
print(payments_df.head())
print('Counts for each type of payment_code value: ')
print(payments_df['payment_code'].value_counts())
print('Counts of records by transaction date: ')
print(payments_df['transaction_date'].value_counts())
print('Summary statistics for payment_amt: ')
print(payments_df['payment_amt'].describe())
print('Counts of businesses by entity_type: ')
print(clients_df['entity_type'].value_counts())
print('Counts of businesses by year established: ')
print(clients_df['entity_year_established'].value_counts())

   transaction_id  contract_id  client_id transaction_date  payment_amt  \
0           20175          927          1          05-2018        66.66   
1            8485          927          1          11-2017        66.66   
2           13778          927          1          02-2018        66.66   
3           22768          927          1          06-2018        66.66   
4           15698          927          1          03-2018        66.66   

  payment_code  
0      PAYMENT  
1      PAYMENT  
2      PAYMENT  
3      PAYMENT  
4      PAYMENT  
Counts for each type of payment_code value: 
payment_code
PAYMENT    23340
DEFAULT     2219
Name: count, dtype: int64
Counts of records by transaction date: 
transaction_date
06-2018    2469
05-2018    2430
04-2018    2272
07-2018    2245
03-2018    1917
02-2018    1905
07-2017    1884
01-2018    1871
08-2017    1759
11-2017    1751
10-2017    1742
12-2017    1661
09-2017    1653
Name: count, dtype: int64
Summary statistics for payment_amt: 
c

In [18]:
# Join the two dataframes into a single dataframe joining on 'client id' column
merged_df = pd.merge(payments_df, clients_df, on='client_id')
merged_df.head()

,transaction_id,contract_id,client_id,transaction_date,payment_amt,payment_code,entity_type,entity_year_established
0,20175,927,1,05-2018,66.66,PAYMENT,Other Partnership,2006
1,8485,927,1,11-2017,66.66,PAYMENT,Other Partnership,2006
2,13778,927,1,02-2018,66.66,PAYMENT,Other Partnership,2006
3,22768,927,1,06-2018,66.66,PAYMENT,Other Partnership,2006
4,15698,927,1,03-2018,66.66,PAYMENT,Other Partnership,2006
